In [1]:
#include <memory>
#include <vector>

#include <iostream>

# **Duck Typing en C++**

<img width="384" height="384" src="https://raw.githubusercontent.com/newlawrence/Talks/master/181122_duck_typing/pictures/duck-prints.svg?sanitize=true">

Image by [svgrepo.com](https://www.svgrepo.com/svg/32836/duck-prints)

### Alberto Lorenzo Márquez

* Ingeniero aeroespacial

* Desarrollador de software (**C++**, **Fortran**, **Python**, **Javascript**)

* **Github**: https://github.com/newlawrence

* **Twitter**: [@newlawrence](https://twitter.com/newlawrence)

# Polimorfismo

* Polimorfismo dinámico

* Polimorfismo estático

## Polimorfismo dinámico

El tradicional basado en **herencia** y las **funciones virtuales**.

In [2]:
class IFoo {
public:
    virtual void bar() const = 0;
    virtual ~IFoo() = default;
};

In [3]:
class Foo1 final : public IFoo {
public:
    void bar() const override {
        std::cout << "I'm a Foo1 instance" << std::endl;
    }
};

class Foo2 final : public IFoo {
public:
    void bar() const override {
        std::cout << "I'm a Foo2 instance" << std::endl;
    }
};

In [4]:
std::vector<std::unique_ptr<IFoo>> foo_ptrs;
foo_ptrs.emplace_back(std::make_unique<Foo1>(Foo1()));
foo_ptrs.emplace_back(std::make_unique<Foo2>(Foo2()));

for (auto const& foo : foo_ptrs)
    foo->bar();

I'm a Foo1 instance
I'm a Foo2 instance


¿Utilizar una clase que no pertenece a la jerarquía?

In [5]:
class Foo3 {
public:
    void bar() const { std::cout << "I'm a Foo3 instance" << std::endl; }
};

### Principales inconvenientes

* Semántica por referencia.

* Requiere de código adicional para adaptar código de terceros.

## Polimorfismo estático

Es decir, las plantillas.

In [6]:
template<typename... Args>
void call_bar(Args&&... foos) { (foos.bar(), ...); }

In [7]:
call_bar(Foo1(), Foo2(), Foo3());

I'm a Foo1 instance
I'm a Foo2 instance
I'm a Foo3 instance


¿Almacenar distantas instancias de clases de plantilla en un contenedor...

### Tupla

... dinámico en tiempo de ejecución...

### Variant

... para cualquier tipo que cumpla la interfaz?

### ¿Any?

### Principal inconveniente

* Comunicación entre distintas unidades de traducción sólo para tipos conocidos.

### Repasando...

...la herencia permite utilizar tipos desconocidos a través de las interfaces.

...las plantillas permiten generar código de forma automática.

## Combinando polimorfismos

In [8]:
class FooConcept {
public:
    virtual void bar() const = 0;
    virtual ~FooConcept() = default;
};

Utilizando la herencia proporcionando una clase base para las plantillas, se consigue que estas no tengan por qué ser conocidas entre distintas unidades de traducción.

In [9]:
template<typename T>
class FooModel final : public FooConcept {
    T self_;

public:
    template<typename U>
    FooModel(U&& self) : self_{ std::forward<U>(self) } {}

    void bar() const override { self_.bar(); }
};

Utilizando las plantillas, se consigue liberar al usuario de tener que escribir código para adaptarse a la interfaz.

## Borrado de tipos

In [10]:
class Foo {
    std::unique_ptr<FooConcept> self_;

public:
    template<typename T>
    Foo(T&& self)
        : self_{ std::make_unique<FooModel<T>>(std::forward<T>(self)) }
    {}

    void bar() const { self_->bar(); }
};

Finalmente, ocultando las dos técnicas anteriores bajo un mismo tipo, se consigue proporcionar semántica por valor.

In [11]:
std::vector<Foo> foos;
foos.emplace_back(Foo1());  // Same hierarchy
foos.emplace_back(Foo2());  // Same hierarchy
foos.emplace_back(Foo3());  // Foreign class

for (auto const& foo : foos)
    foo.bar();

I'm a Foo1 instance
I'm a Foo2 instance
I'm a Foo3 instance


### Duck Typing

> When I see a bird that walks like a duck and swims like a duck and quacks like a duck, I call that bird a duck.
>
> <cite><b>James Whitcomb Riley</b></cite>

# Algo de control sobre la memoria dinámica

***Optimización del objeto pequeño***

***Recursos de memoria polimórficos***

## Tablas de métodos virtuales

Los mecanismos de implementación de la optimización del objeto pequeño y reserva de memoria de objetos con tipos borrados requieren de castings intensivos.

Las clases con métodos virtuales no poseen una disposición estándar (**standard layout**). Los castings directos de clase base a una localización de memoria donde se ha almacenado una clase derivada conducen a **comportamiento indefinido**.

De la herencia a las tablas de métodos... ¿cómo cambia el borrado de tipos?

```c++
class FooWrapper {
public:
    using allocator_type = std::pmr::polymorphic_allocator<std::byte>;

private:
    static std::size_t constexpr BUFFER_SIZE = 16;

    allocator_type allocator_;

    FooConcept const* dispatcher_;
    union Storage {
        std::aligned_storage_t<BUFFER_SIZE> local;
        void* remote;
    } storage_;

    // ...
};
```

## Conceptos como tablas virtuales de punteros

```c++
struct FooConcept {
    void(*bar)(void* self);
    void(*destroy)(void* self, void* alloc);
};
```

Implementando la tabla de punteros para los objetos pequeños...

```c++
template<typename T, typename U>
FooConcept const FooDispatcherSBO{
    [](void* self) {         // bar
        auto storage = reinterpret_cast<T*>(&static_cast<U>(self)->local);
        storage->bar();
    },
    [](void* self, void*) {  // destroy
        auto storage = reinterpret_cast<T*>(&static_cast<U>(self)->local);
        std::destroy_at(storage);
    }
};
```

Implementando la tabla de punteros para los objetos utilizando memoria dinámica...

```c++
template<typename T, typename U>
FooConcept const FooDispatcherPMR{
    [](void* self) {               // bar
        auto storage = reinterpret_cast<T*>(&static_cast<U>(self)->remote);
        remote->bar();
    },
    [](void* self, void* alloc) {  // destroy
        auto allocator = detail::rebind_function<T>(alloc);
        auto storage = reinterpret_cast<T*>(&static_cast<U>(self)->remote);
        if (storage)
            std::destroy_at(storage);
        allocator.deallocate(storage, 1);
    }
};
```

## Borrado de tipos

```c++
class FooWrapper {
public:
    template<typename T>
    explicit FooWrapper(T&& obj, allocator_type alloc={})
        : allocator_{ alloc }
    {
        std::pmr::polymorphic_allocator<T> allocator{ alloc };
        T* storage;

        if constexpr (sizeof(T) <= BUFFER_SIZE) {
            dispatcher_ = &FooDispatcherSBO<T, Storage>;
            storage = reinterpret_cast<T*>(&storage_.local);
        }
        else {
            dispatcher_ = &FooDispatcherPMR<T, Storage>;
            storage_.remote = allocator.allocate(1);
            storage = reinterpret_cast<T*>(storage_.remote);
        }
        allocator.construct(storage, std::forward<T>(obj));
    }
```

El desctructor y el método `bar` son despachados a través de la tabla de punteros:

```c++
FooWrapper::~FooWrapper() {
    dispatcher_->destroy(&storage_, &allocator_);
}
```

```c++
void FooWrapper::bar() const {
    dispatcher_->bar(&const_cast<Storage&>(storage_));
}
```

## Utilizando un bloque de memoria local

Definiendo un objeto pequeño:

```c++
class SmallFoo {
public:
    void bar() const { std::cout << "I am a small foo!" << std::endl; }
};
```

Definiendo un objeto que requiera de memoria dinámica:

```c++
class BigFoo {
    char str_[24];

public:
    BigFoo() noexcept { std::strcpy(str_, "I am a big foo!"); }
    BigFoo(BigFoo const& other) noexcept : BigFoo{} {}
    BigFoo(BigFoo&& other) noexcept : BigFoo{} {}

    void bar() const { std::cout << str_ << std::endl; }
};
```

Ejemplo envolviendo ambos objetos en un vector que se aprovecha de un bloque de memoria local:

```c++
std::array<std::byte, 256> buffer;
std::pmr::monotonic_buffer_resource pool{ buffer.data(), buffer.size() };

std::pmr::vector<foo::FooWrapper> foos{ &pool };
foos.emplace_back(SmallFoo());
foos.emplace_back(BigFoo());

for (auto const& foo : foos)
    foo.bar();
```

Demostrado está, en **C++** es posible un polimorfismo...

* Con semántica por valor.

* Sin necesidad de aportar código adicional.

* Con control sobre la gestión de la memoria.

### ... por eso nos gusta C++!

# Código fuente

El ejemplo correcto y completo de borrado de tipos aquí presentado, utilizando optimización del objeto pequeño y recursos de memoria polimórficos (incluyendo los constructores de copia y movimiento, así como los operadores de asignación), puede encontrarse [aquí](https://github.com/newlawrence/Talks/tree/master/181122_duck_typing/source).

# Referencias

Dave Kilian, [**C++ 'Type Erasure' Explained**](http://davekilian.com/cpp-type-erasure.html)

José Daniel García, [Polimorfismo estático y dinámico en C++11: ¿Flexibilidad contra rendimiento?](https://www.youtube.com/watch?v=yxDPlWr2WD0&t=1765s)

Sean Parent, [Polymorphic Task](https://www.youtube.com/watch?v=2KGkcGtGVM4)

Sean Parent, [Small Object Optimization for Polymorphic Types](http://stlab.cc/tips/small-object-optimizations.html)

Louis Dionne, [Runtime Polymorphism: Back to the Basics](https://www.youtube.com/watch?v=gVGtNFg4ay0&t=3063s)